In [53]:
import logging

In [8]:
import json

with open('trudkod.json') as f:
    doc = json.load(f)

In [9]:
articles = doc['articles']

In [2]:
pip install transformers

     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/134.8 kB ? eta -:--:--
     ----------- ------------------------- 41.0/134.8 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 134.8/134.8 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ----------------------------- ---------- 30.7/42.0 kB ? eta -:--:--
     -------------------------------------- 42.0/42.0 kB 511.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.8 MB 825.8 kB/s eta 0:00:11
    --------------------------------------- 0.1/8.8 MB 1.2 MB/s eta 0:00:08
   - -------------------------------------- 0.3/8.8 MB 1.5 MB/s eta 0:00:06
   - -----------------------

In [1]:
from transformers import AutoTokenizer

model_name = 'mistralai/Mistral-7B-v0.1'

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example sentence to tokenize
sentence = articles[0]['text']

# Tokenize the sentence
tokens = tokenizer(sentence, return_tensors='pt')


ModuleNotFoundError: No module named 'transformers'

In [10]:
def num_of_tokens(text):
    tokens = tokenizer(text, return_tensors='pt')
    num_tokens = tokens['input_ids'].size(1)
    return num_tokens

In [13]:
import pandas as pd
df = pd.DataFrame(articles)
df['full_text'] = df.path + ".\n##" + df.article_name + '.\n' + df.text
df['num_tokens'] = df.full_text.map(num_of_tokens)

In [145]:
prompt_template = """1. Imagine in which real-life situations the information below might be useful, make up 3 CASES that describe some situation and QUESTION related to this text. In question dont mention source of information
2. On behalf of an experienced lawyer, write answers to these questions and prove your point of view with quotes from original document ( ... "quote" ...).
Write everything in Russian.
Document:
'{document}'
Example:
C: Работник подал в суд на работодателя, утверждая, что трудовые отношения с ним были незаконно прекращены. Работодатель утверждает, что не существует заключенного трудового договора.
Q: Может ли трудовое отношение считаться заключенным в случае, если трудовой договор не был надлежащим образом оформлен, но работник был фактически допущен к работе?
A: Да, трудовое отношение может считаться заключенным, даже если трудовой договор не был надлежащим образом оформлен, при условии фактического допущения работника к работе с ведома или по поручению работодателя, как указано в статье 16 Трудового кодекса РФ: "Трудовые отношения между работником и работодателем возникают также на основании фактического допущения работника к работе с ведома или по поручению работодателя...".

Follow the format, don`t include in case section the question.

С1: <case 1 description in 1 paragraph, without question>
Q1: <question 1>
A1: <answer 1>

С2: <case 2 description in 1 paragraph, without question>
Q2: <question 2>
A2: <answer 2>

С3: <case 3 description in 1 paragraph, without question>
Q3: <question 3>
A3: <answer 3>

Try to be as heplful as you can, i will tip you if you can help"""

In [67]:
prompt_template2 = """You will be provided with law related document, using this document make up 3 cases that describe some real-life situation and write QUESTION related to this document and answer it, follow instructions for each clause:
Context: Description for made-up plausible situation, mention who and why want to recive lawyer`s help, include important details, dont include question here. 
Question: Ask question based on that context
Answer: On behalf of an experienced lawyer, write answers to these questions, 
Answer format should be following: '[short answer], [explanation]
[proving a point by QUOTE from the document]: "[place quote here]" ([place reference to source here] Трудовой Кодекс РФ)'
Document:
'{document}'
Follow the format, don`t include in case section the question.
Ensure you have quote from document in answer.

C1: <case 1 description>
Q1: <question 1>
A1: <answer 1>

C2: <case 2 description>
Q2: <question 2>
A2: <answer 2>

C3: <case 3 description>
Q3: <question 3>
A3: <answer 3>
Write in Russian.
Try to be as heplful as you can, i will tip you if you can help"""

In [59]:
token_lenght_promt_template = num_of_tokens(prompt_template)

In [60]:
df.iloc[100, 5]

'Часть 3 / Раздел 3 / Глава 14 / Статья 86.\n##Общие требования при обработке персональных данных работника и гарантии их защиты.\n\nВ целях обеспечения прав и свобод человека и гражданина работодатель и его представители при обработке персональных данных работника обязаны соблюдать следующие общие требования:\n1) обработка персональных данных работника может осуществляться исключительно в целях обеспечения соблюдения законов и иных нормативных правовых актов, содействия работникам в трудоустройстве, получении образования и продвижении по службе, обеспечения личной безопасности работников, контроля количества и качества выполняемой работы и обеспечения сохранности имущества;\n2) при определении объема и содержания обрабатываемых персональных данных работника работодатель должен руководствоваться Конституцией Российской Федерации, настоящим Кодексом и иными федеральными законами;\n3) все персональные данные работника следует получать у него самого. Если персональные данные работника воз

In [77]:
print(response)

С1: Кейс 1 - Отказ работника от предоставления персональных данных
Q1: Каковы права работодателя в случае, если работник отказывается предоставить свои персональные данные?
A1: В соответствии со статьей 86 работодатель не имеет права получать и обрабатывать персональные данные работника без его письменного согласия. Таким образом, если работник отказывается предоставить свои персональные данные, работодатель не может требовать их получения.

С2: Кейс 2 - Защита персональных данных работника
Q2: Какой порядок защиты персональных данных работника предусмотрен законодательством?
A2: Согласно статье 86 работодатель обязан обеспечить защиту персональных данных работника от неправомерного использования или утраты за счет своих средств. Это означает, что работодатель должен принимать меры и средства защиты данных работника.

С3: Кейс 3 - Получение персональных данных работника от третьей стороны
Q3: В каких случаях работодатель может получать персональные данные работника от третьей стороны?


In [24]:
from llama_index.llms import Ollama

llm = Ollama(model="mixtral:8x7b-instruct-v0.1-q4_K_M")

def get_answer(prompt_template, document, openai):
    prompt = prompt_template.format(document=document)
    if num_of_tokens(prompt) > 4000:
        return "Контекст превысил лимит\nQ1:Контекст превысил лимит"
    # вопрос к ллм
    if openai:
        response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt.strip('[INST]').strip('[/INST]')}
            ]
        ).choices[0].message.content
    else:
        response = llm.complete(prompt, timeout=600)
#     response = """C1: В компании произошел конфликт между работодателем и работниками относительно условий труда. Работники ссылаются на коллективный договор и требуют улучшения условий, основываясь на его положениях.

# Q1: Какие конкретные аспекты условий труда могут быть регулированы коллективным договором, и каким образом он влияет на улучшение условий для работников?

# A1: В соответствии с коллективным договором, стороны могут регулировать формы, системы и размеры оплаты труда, улучшение условий и охрану труда, а также другие аспекты трудовых отношений. "Содержание и структура коллективного договора определяются сторонами" (цитата из документа).

# C2: Работники высказали обеспокоенность тем, что не получают достаточной компенсации за переработку, несмотря на явные указания в коллективном договоре относительно оплаты труда и рабочего времени.

# Q2: Как коллективный договор регулирует оплату труда и переработку, и могут ли работники требовать дополнительной компенсации на основе этих положений?

# A2: Согласно коллективному договору, оплата труда и рабочее время регулируются в нем. "Формы, системы и размеры оплаты труда; рабочее время и время отдыха" - вопросы, регулируемые коллективным договором (цитата из документа).

# C3: Компания столкнулась с финансовыми трудностями, и работодатель хочет пересмотреть условия коллективного договора, чтобы снизить нагрузку на бюджет.

# Q3: Может ли работодатель изменять условия коллективного договора в связи с финансово-экономическим положением компании, и какие льготы могут быть предоставлены работникам при таких изменениях?

# A3: В соответствии с коллективным договором, при учете финансово-экономического положения работодателя, могут устанавливаться льготы и преимущества для работников. "В коллективном договоре с учетом финансово-экономического положения работодателя могут устанавливаться льготы и преимущества для работников" (цитата из документа)."""
    return response
    

In [95]:
max_tokens = 4000
token_lenght_promt_template = num_of_tokens(prompt_template)
max_document_tokens = max_tokens - token_lenght_promt_template
def cut_text_by_max_number_of_tokens(df, max_tokens=max_document_tokens, col='full_text', margin=100):
    def cut(row):
        if  (n_tokens:=row['num_tokens']) > 3500:
            mean_word_len = len(row['full_text'].split(" ")) / n_tokens
            return " ".join(row['full_text'].split(" ")[:int((max_tokens - margin) * mean_word_len)])
        else:
            return row['full_text']
    
    df.full_text = df.apply(cut, axis=1)
cut_text_by_max_number_of_tokens(df)
df.full_text.map(num_of_tokens).max()

3622

In [3]:
import json

def parse_output_to_dict(generated_text):
    case_question_answers = {}
    for line in generated_text.split("\n"):
        # Assuming the model was instructed to format output in a "C#: Case" "Q#: Question" and "A#: Answer" pattern
        if (line.startswith("Q") or line.startswith("A") or line.startswith("C") or line.startswith("С")) and ":" in line:
            # Extract question number and text
            letter_num, question = line.split(":", 1)
            case_question_answers[letter_num.strip()] = [question.strip()]

    return case_question_answers


In [16]:
n = 45
df[n:n+1]

,article_name,text,path,section_name,chapter_name,full_text,num_tokens
45,Содержание и структура коллективного договора,\nСодержание и структура коллективного договор...,Часть 2 / Раздел 2 / Глава 7 / Статья 41,Социальное партнерство в сфере труда,Коллективные договоры и соглашения,Часть 2 / Раздел 2 / Глава 7 / Статья 41.\n##С...,644


In [51]:
(df.num_tokens.sum() / 1000 * 0.0005 + 500 * 0.0015 / 1000 * 500 ) * 100

52.6006

In [49]:
num_of_tokens(" ".join(pd.read_json('gpt01.jsonl', lines=True).loc[0, :"A3"]))

485

In [99]:
pd.read_json('gpt01.jsonl', lines=True)

,Q1,A1,Q2,A2,Q3,A3,index,context,openai
0,Какие льготы и преимущества могут быть установ...,В коллективном договоре с учетом финансово-эко...,Какие вопросы в области условий и охраны труда...,В коллективном договоре могут быть определены ...,Какие гарантии и интересы работников могут быт...,В коллективном договоре могут быть определены ...,45,Часть 2 / Раздел 2 / Глава 7 / Статья 41.\n##С...,True


In [35]:
for i in .iloc[0, :]:
    print(i)

Какие льготы и преимущества могут быть установлены для работников через коллективный договор?
В коллективном договоре с учетом финансово-экономического положения работодателя могут устанавливаться льготы и преимущества для работников, условия труда, более благоприятные по сравнению с установленными законами, иными нормативными правовыми актами, соглашениями. ("В коллективном договоре с учетом финансово-экономического положения работодателя могут устанавливаться льготы и преимущества для работников...")
Какие вопросы в области условий и охраны труда могут быть включены в коллективный договор?
В коллективном договоре могут быть определены вопросы улучшения условий и охраны труда работников, в том числе женщин и молодежи, а также экологическая безопасность и охрана здоровья работников на производстве. ("улучшение условий и охраны труда работников, в том числе женщин и молодежи;", "экологическая безопасность и охрана здоровья работников на производстве;")
Какие гарантии и интересы работник

In [20]:
# get file size in python
import os

file_name = "CaseQuestionsAnswer.jsonl"

file_stats = os.stat(file_name)

print(file_stats)
print(f'File Size in Bytes is {file_stats.st_size}')
print(f'File Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

os.stat_result(st_mode=33204, st_ino=57934891, st_dev=66309, st_nlink=1, st_uid=1000, st_gid=1000, st_size=9483012, st_atime=1706878345, st_mtime=1706878343, st_ctime=1706878343)
File Size in Bytes is 9483012
File Size in MegaBytes is 9.043704986572266


In [4]:
OPENAI_API_KEY = "..."

In [47]:
def format_prompt(content):
    return prompt_template2.format(document=content)

In [64]:
import time
import asyncio
import aiohttp
from openai import AsyncOpenAI

client = AsyncOpenAI(api_key=OPENAI_API_KEY)
class ProgressLog:
    def __init__(self, total):
        self.total = total
        self.done = 0

    def increment(self):
        self.done = self.done + 1

    def __repr__(self):
        return f"Done runs {self.done}/{self.total}."

async def get_completion(index, row, semaphore, progress_log, filename):
    async with semaphore:

        await asyncio.sleep(1)
        content = format_prompt(row['full_text'])
        generated_text = await client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": content}],
            temperature=0.3)
        
        progress_log.increment()
        print(progress_log)
        parsed_response = parse_output_to_dict(generated_text.choices[0].message.content.strip())
        parsed_response['prompt'] = content
        parsed_response['index'] = index
        parsed_response['prompt'] = content
        parsed_response['generated_text'] = generated_text.choices[0].message.content
    
        line = pd.DataFrame.from_dict(parsed_response).to_json(orient='records', lines=True)

        with open(f"{filename}.jsonl", 'a') as f:
            f.write(line)
        return generated_text.choices[0].message.content

async def get_completion_list(df, max_parallel_calls, filename):
    semaphore = asyncio.Semaphore(value=max_parallel_calls)
    progress_log = ProgressLog(len(df))

    async with aiohttp.ClientSession() as session:
        result = []
        for index, row in df.iterrows():
            result.append(get_completion(index, row, semaphore, progress_log, filename))
        return await asyncio.gather(*result)

# start_time = time.perf_counter()
# completion_list = await get_completion_list(["ping", "pong"]*5, 100)
# print("Time elapsed: ", time.perf_counter() - start_time, "seconds.")
# print(completion_list)

In [25]:
df.iloc[100, 5]

'Часть 3 / Раздел 3 / Глава 14 / Статья 86.\n##Общие требования при обработке персональных данных работника и гарантии их защиты.\n\nВ целях обеспечения прав и свобод человека и гражданина работодатель и его представители при обработке персональных данных работника обязаны соблюдать следующие общие требования:\n1) обработка персональных данных работника может осуществляться исключительно в целях обеспечения соблюдения законов и иных нормативных правовых актов, содействия работникам в трудоустройстве, получении образования и продвижении по службе, обеспечения личной безопасности работников, контроля количества и качества выполняемой работы и обеспечения сохранности имущества;\n2) при определении объема и содержания обрабатываемых персональных данных работника работодатель должен руководствоваться Конституцией Российской Федерации, настоящим Кодексом и иными федеральными законами;\n3) все персональные данные работника следует получать у него самого. Если персональные данные работника воз

In [101]:
len(df), [i * 30 for i in range(9, 18)]

(529, [270, 300, 330, 360, 390, 420, 450, 480, 510])

[9, 10, 11, 12, 13, 14, 15, 16, 17]

In [102]:
for i in range(9, 18):
    start_time = time.perf_counter()
    start = 30*i
    end = start + 30
    completion_list = await get_completion_list(df.iloc[start:end, :], 100, filename=f"batch{start}-{end-1}")
    print("Time elapsed: ", time.perf_counter() - start_time, "seconds.")

Done runs 1/30.
Done runs 2/30.
Done runs 3/30.
Done runs 4/30.
Done runs 5/30.
Done runs 6/30.
Done runs 7/30.
Done runs 8/30.
Done runs 9/30.
Done runs 10/30.
Done runs 11/30.
Done runs 12/30.
Done runs 13/30.
Done runs 14/30.
Done runs 15/30.
Done runs 16/30.
Done runs 17/30.
Done runs 18/30.
Done runs 19/30.
Done runs 20/30.
Done runs 21/30.
Done runs 22/30.
Done runs 23/30.
Done runs 24/30.
Done runs 25/30.
Done runs 26/30.
Done runs 27/30.
Done runs 28/30.
Done runs 29/30.
Done runs 30/30.
Time elapsed:  97.22266760002822 seconds.
Done runs 1/30.
Done runs 2/30.
Done runs 3/30.
Done runs 4/30.
Done runs 5/30.
Done runs 6/30.
Done runs 7/30.
Done runs 8/30.
Done runs 9/30.
Done runs 10/30.
Done runs 11/30.
Done runs 12/30.
Done runs 13/30.
Done runs 14/30.
Done runs 15/30.
Done runs 16/30.
Done runs 17/30.
Done runs 18/30.
Done runs 19/30.
Done runs 20/30.
Done runs 21/30.
Done runs 22/30.
Done runs 23/30.
Done runs 24/30.
Done runs 25/30.
Done runs 26/30.
Done runs 27/30.
Done ru

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 4253 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

Done runs 1/19.
Done runs 2/19.
Done runs 3/19.
Done runs 4/19.
Done runs 5/19.
Done runs 6/19.
Done runs 7/19.
Done runs 8/19.
Done runs 9/19.
Done runs 10/19.
Done runs 11/19.
Done runs 12/19.
Done runs 13/19.
Done runs 14/19.
Done runs 15/19.
Done runs 16/19.
Done runs 17/19.
Done runs 18/19.


In [69]:
print(completion_list[10])

C1: Владимир работает в частной компании в качестве программиста уже несколько лет. Он обратился к адвокату, так как компания не выплачивает ему зарплату вовремя и не предоставляет отпускные. Владимир хочет узнать, могут ли применяться положения трудового законодательства и иных актов, содержащих нормы трудового права, к его ситуации.

Q1: Могут ли применяться положения трудового законодательства и иных актов, содержащих нормы трудового права, к отношениям между Владимиром и его работодателем?
A1: Да, положения трудового законодательства и иных актов, содержащих нормы трудового права, применяются к трудовым отношениям и иным непосредственно связанным с ними отношениям. "Все работодатели в трудовых отношениях и иных непосредственно связанных с ними отношениях с работниками обязаны руководствоваться положениями трудового законодательства и иных актов, содержащих нормы трудового права" (Трудовой Кодекс РФ, Статья 11).

C2: Анна работает в крупной компании в качестве менеджера по продажам.

In [86]:
res_df = pd.read_json('batch210-239.jsonl', lines=True)

In [82]:
def format_output(row):
    for i in range(1, 4):
        for col_symb in ['C', 'С', 'Q', 'A']:
            try:
                print(f'{col_symb}{i}', row[f'{col_symb}{i}'])
            except KeyError:
                print(f'{col_symb}{i}'+'_')

In [96]:
df.full_text.str.len().nlargest()

241    10065
25      8903
526     7728
453     7541
94      7530
Name: full_text, dtype: int64

5478

In [59]:
format_output(res_df.iloc[0, :])

C1 В компании "Альфа" работает сотрудник по имени Иван. Работодатель хранит и использует его персональные данные в соответствии с требованиями закона. Однако, Иван обнаружил, что его данные были переданы третьим лицам без его согласия. Он обратился к адвокату с просьбой о помощи.
С1_
Q1 Какие требования устанавливаются работодателем при хранении и использовании персональных данных работников?
A1 Порядок хранения и использования персональных данных работников устанавливается работодателем с соблюдением требований настоящего Кодекса и иных федеральных законов. (ТК РФ, Часть 3, Раздел 3, Глава 14, Статья 87)
C2 В компании "Бета" работает сотрудник по имени Анна. Работодатель хранит и использует ее персональные данные, но не соблюдает требования закона. Анна решила обратиться к адвокату, чтобы узнать свои права и возможные меры защиты.
С2_
Q2 Что делать, если работодатель не соблюдает требования закона при хранении и использовании персональных данных работников?
A2 В случае нарушения работ

In [83]:
[(index, row['full_text'])for index, row in df.iloc[5:10, :].iterrows()]

[(5,
  'Часть 1 / Раздел 1 / Глава 1 / Статья 6.\n##Разграничение полномочий между федеральными органами государственной власти и органами государственной власти субъектов Российской Федерации в сфере трудовых отношений и иных непосредственно связанных с ними отношений.\n\nК ведению федеральных органов государственной власти в сфере трудовых отношений и иных непосредственно связанных с ними отношений относится принятие обязательных для применения на всей территории Российской Федерации федеральных законов и иных нормативных правовых актов, устанавливающих:\nосновные направления государственной политики в сфере трудовых отношений и иных непосредственно связанных с ними отношений;\nосновы правового регулирования трудовых отношений и иных непосредственно связанных с ними отношений (включая определение правил, процедур, критериев и нормативов, направленных на сохранение жизни и здоровья работников в процессе трудовой деятельности);\nобеспечиваемый государством уровень трудовых прав, свобод

In [23]:
print(df.loc[150, 'full_text'])

Часть 3 / Раздел 6 / Глава 21 / Статья 135.
##Установление заработной платы.

Заработная плата работнику устанавливается трудовым договором в соответствии с действующими у данного работодателя системами оплаты труда.
Системы оплаты труда, включая размеры тарифных ставок, окладов (должностных окладов), доплат и надбавок компенсационного характера, в том числе за работу в условиях, отклоняющихся от нормальных, системы доплат и надбавок стимулирующего характера и системы премирования, устанавливаются коллективными договорами, соглашениями, локальными нормативными актами в соответствии с трудовым законодательством и иными нормативными правовыми актами, содержащими нормы трудового права.
Российская трехсторонняя комиссия по регулированию социально-трудовых отношений ежегодно до внесения в Государственную Думу Федерального Собрания Российской Федерации проекта федерального закона о федеральном бюджете на очередной финансовый год и плановый период разрабатывает единые рекомендации по установл

In [137]:
import glob

folder_path = ''
file_mask = 'batch*'  # Change this to your desired file mask

matching_files = glob.glob(f"{folder_path}{file_mask}")
concated_df = pd.concat([pd.read_json(file_name, lines=True).set_index('index') for file_name in matching_files]).sort_index()

In [129]:
long_df = pd.DataFrame().reindex_like(concated_df)
long_df.drop([f'{col_symb}{i}' for col_symb in ['C', 'Q', 'A'] for i in range(1,4)], axis=1)

,prompt,generated_text,Quote,Согласно статье 133.1 Трудового Кодекса РФ,Ситуация
index,,,,,
25,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,NaN,NaN,NaN
28,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
118,NaN,NaN,NaN,NaN,NaN
107,NaN,NaN,NaN,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN


In [115]:
for i in range(1, 4):
    long_df

(527, 15)

In [130]:
concated_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 527 entries, 25 to 104
Data columns (total 14 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   C1                                          527 non-null    object
 1   Q1                                          526 non-null    object
 2   prompt                                      527 non-null    object
 3   generated_text                              527 non-null    object
 4   A1                                          525 non-null    object
 5   C2                                          525 non-null    object
 6   Q2                                          524 non-null    object
 7   A2                                          524 non-null    object
 8   C3                                          522 non-null    object
 9   Q3                                          520 non-null    object
 10  A3                            

In [134]:
print(concated_df.dropna(subset=['Quote']).generated_text[19])

C1: Василий работает на строительной фирме по гражданско-правовому договору. Он выполняет работы по ремонту зданий и сооружений, но не имеет трудового договора с компанией. Василий решил обратиться за помощью к адвокату, чтобы признать его отношения с компанией трудовыми отношениями.

Q1: Может ли Василий обратиться в суд для признания его отношений с компанией трудовыми отношениями?

A1: Да, Василий может обратиться в суд для признания его отношений с компанией трудовыми отношениями. Согласно Трудовому Кодексу РФ, в случае прекращения отношений, связанных с использованием личного труда и возникших на основании гражданско-правового договора, признание этих отношений трудовыми отношениями осуществляется судом. Физическое лицо, являвшееся исполнителем по указанному договору, вправе обратиться в суд за признанием этих отношений трудовыми отношениями в порядке и в сроки, которые предусмотрены для рассмотрения индивидуальных трудовых споров.

Quote: "В случае прекращения отношений, связанны

In [264]:
def text_into_dict(text):
    keys = [f'{col_symb}{i}' for col_symb in ['C', 'Q', 'A'] for i in range(1,4)]
    text.replace("Quote", 'Цитата').replace("quote",'цитата')
    case_question_answers = {}
    for line in text.split("\n"):
        # Assuming the model was instructed to format output in a "C#: Case" "Q#: Question" and "A#: Answer" pattern
        if line[:2] in keys and ":" in line:
            # Extract question number and text
            key, value = line.split(":", 1)
            case_question_answers[key.strip()] = value.strip()
        else:
            pattern = re.compile(r'[a-zA-Z\[\]]')
            if line.strip() != '':
                if pattern.search(line):
                    if 'other_text' in case_question_answers.keys():
                        case_question_answers['other_text'] += "\n"
                    else:
                        case_question_answers['other_text'] = ''
                    case_question_answers['other_text'] += line.strip()
                else:
                    case_question_answers[key.strip()] +="\n" + line.strip()
    return case_question_answers

In [179]:
prompt_len_prefix = len(prompt_template2.split('{')[0])
prompt_len_postfix = len(prompt_template2.split('}', maxsplit=1)[1])
concated_df['document'] = concated_df.prompt.map(lambda s: s[prompt_len_prefix:-(prompt_len_postfix+1)])

In [180]:
concated_df.generated_text.map()

index
0      C1: Василий работает в крупной компании и был ...
1      C1: Василий работает в крупной компании и заме...
2      C1: В компании "Альфа" работает женщина по име...
3      C1: В компании "Альфа" работник Иванов Алексей...
4      C1: Василий работает в крупной компании и стол...
                             ...                        
523    C1: В компании "Альфа" работник по имени Ивано...
524    C1: Василий работает в компании, которая была ...
525    C1: Владимир работает на стройке уже несколько...
527    C1: Владимир работает в крупной компании и явл...
528    C1: Василий работает в компании "Альфа" с 2010...
Name: generated_text, Length: 527, dtype: object

,C1,Q1,A1,C2,Q2,A2,C3,Q3,A3,Quote,other_text,Согласно статье 133.1 Трудового Кодекса РФ,Ситуация
index,,,,,,,,,,,,,
0,Василий работает в крупной компании и был увол...,Является ли увольнение Василия без объяснения ...,"Нет, увольнение Василия без объяснения причин ...",Анна работает в небольшой фирме и не получает ...,Какие права имеет Анна в случае несвоевременно...,Анна имеет право на своевременную выплату зара...,"Иван работает на предприятии, где нарушаются у...",Какие меры можно принять в случае нарушения ус...,"В случае нарушения условий труда, включая недо...",NaN,NaN,NaN,NaN
1,"Василий работает в крупной компании и заметил,...",Есть ли у Василия право на равную оплату труда...,"Да, у Василия есть право на равную оплату труд...",Анна работает в небольшой компании и ей не пре...,Обязан ли работодатель предоставить Анне оплач...,"Да, работодатель обязан предоставить Анне опла...","Иван работает в компании, где его начальник по...",Как Иван может защитить свое достоинство и пра...,Иван может защитить свое достоинство и права о...,NaN,NaN,NaN,NaN
2,"В компании ""Альфа"" работает женщина по имени А...",Может ли Анна обратиться в суд по поводу отказ...,"Да, Анна имеет право обратиться в суд с заявле...","В компании ""Бета"" работает мужчина по имени Ив...",Может ли Иван обратиться в суд по поводу возмо...,"Да, Иван имеет право обратиться в суд с заявле...","В компании ""Гамма"" работает мужчина по имени П...",Может ли Петр обратиться в суд по поводу возмо...,"Да, Петр имеет право обратиться в суд с заявле...",NaN,NaN,NaN,NaN
3,"В компании ""Альфа"" работник Иванов Алексей Але...",Может ли Иванов обратиться в суд за нарушение ...,"Да, Иванов может обратиться в суд за нарушение...","В компании ""Бета"" работнику Петрову Ивану Васи...",Может ли Петров отказаться от выполнения работ...,"Да, Петров имеет право отказаться от выполнени...","В компании ""Гамма"" работнику Сидорову Петру Ни...",Может ли Сидоров отказаться от выполнения рабо...,"Да, Сидоров имеет право отказаться от выполнен...",NaN,NaN,NaN,NaN
4,Василий работает в крупной компании и столкнул...,Какие нормативные правовые акты регулируют тру...,Трудовые отношения и иные непосредственно связ...,Анна работает в маленькой компании и ей отказы...,"Что делать, если работодатель отказывает в опл...",В случае отказа работодателя в оплате сверхуро...,"Иван работает на предприятии, где не соблюдают...",Какие нормативные правовые акты регулируют охр...,Охрана труда регулируется трудовым законодател...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,"В компании ""Альфа"" работник по имени Иванов бы...",Какие виды ответственности могут быть применен...,Работодатель может быть привлечен к дисциплина...,"В ресторане ""Гурман"" работница по имени Петров...",Какие виды ответственности могут быть применен...,Начальник может быть привлечен к дисциплинарно...,"В компании ""Бета"" работник по имени Сидоров бы...",Какие виды ответственности могут быть применен...,Работодатель может быть привлечен к дисциплина...,NaN,"[Документ]: ""Лица, виновные в нарушении трудов...",NaN,NaN
524,"Василий работает в компании, которая была созд...",Какие сроки введения в действие настоящего Код...,"В случае, если компания была создана до 1 февр...","Екатерина работает в новой компании, которая б...",Какие сроки введения в действие настоящего Код...,"В случае, если компания была создана после 1 ф...","Андрей работает в компании, которая была созда...",Какие сроки введения в действие настоящего Код...,"В случае, если компания была создана до 1 февр...",NaN,NaN,NaN,NaN
525,Владимир работает на стройке уже несколько лет...,Какие порядок и сроки повышения минимального р...,Порядок и сроки поэтапного повышения минимальн...,"Анна работает в крупной компании и заметила, ч...",Может ли Анна требовать повышения своей оплаты...,В соответствии с частью первой статьи 133 Труд...,"Денис работает на предприятии, которое находит...",Может ли работодатель снизить оплату труда раб...,Согласно част

In [295]:
new_columns_df = concated_df.generated_text.apply(text_into_dict).apply(pd.Series)
# Concatenate the new columns with the original DataFrame
result_df = pd.concat([concated_df[['document', 'generated_text']], new_columns_df], axis=1)

In [277]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 456 entries, 0 to 528
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   document        456 non-null    object
 1   generated_text  456 non-null    object
 2   C1              456 non-null    object
 3   Q1              456 non-null    object
 4   A1              456 non-null    object
 5   C2              456 non-null    object
 6   Q2              456 non-null    object
 7   A2              456 non-null    object
 8   C3              456 non-null    object
 9   Q3              456 non-null    object
 10  A3              456 non-null    object
dtypes: object(11)
memory usage: 42.8+ KB


In [288]:
test_df = pd.DataFrame([range(10), range(10, 20), range(20, 30)]).transpose()
test_df.columns = list('abcde')
test_df.columns

,a,b,c
0,0,10,20
1,1,11,21
2,2,12,22
3,3,13,23
4,4,14,24
5,5,15,25
6,6,16,26
7,7,17,27
8,8,18,28
9,9,19,29


In [307]:
def melt_modified(df: pd.DataFrame, common_columns, columns_to_reformat, new_names):
    df  = df.copy()
    index_name = df.index.name
    common_columns.append(index_name)
    df.reset_index(inplace=True)
    lst_of_tables = []
    for set_of_columns in columns_to_reformat:
        temp_table = df[common_columns+set_of_columns]
        temp_table.columns = common_columns + new_names
        lst_of_tables.append(temp_table)
    return pd.concat(lst_of_tables).reset_index(drop=True)

In [290]:
result_df

,document,generated_text,C1,Q1,A1,C2,Q2,A2,C3,Q3,A3
index,,,,,,,,,,,
0,Часть 1 / Раздел 1 / Глава 1 / Статья 1.\n##Це...,C1: Василий работает в крупной компании и был ...,Василий работает в крупной компании и был увол...,Является ли увольнение Василия без объяснения ...,"Нет, увольнение Василия без объяснения причин ...",Анна работает в небольшой фирме и не получает ...,Какие права имеет Анна в случае несвоевременно...,Анна имеет право на своевременную выплату зара...,"Иван работает на предприятии, где нарушаются у...",Какие меры можно принять в случае нарушения ус...,"В случае нарушения условий труда, включая недо..."
1,Часть 1 / Раздел 1 / Глава 1 / Статья 2.\n##Ос...,C1: Василий работает в крупной компании и заме...,"Василий работает в крупной компании и заметил,...",Есть ли у Василия право на равную оплату труда...,"Да, у Василия есть право на равную оплату труд...",Анна работает в небольшой компании и ей не пре...,Обязан ли работодатель предоставить Анне оплач...,"Да, работодатель обязан предоставить Анне опла...","Иван работает в компании, где его начальник по...",Как Иван может защитить свое достоинство и пра...,Иван может защитить свое достоинство и права о...
2,Часть 1 / Раздел 1 / Глава 1 / Статья 3.\n##За...,"C1: В компании ""Альфа"" работает женщина по име...","В компании ""Альфа"" работает женщина по имени А...",Может ли Анна обратиться в суд по поводу отказ...,"Да, Анна имеет право обратиться в суд с заявле...","В компании ""Бета"" работает мужчина по имени Ив...",Может ли Иван обратиться в суд по поводу возмо...,"Да, Иван имеет право обратиться в суд с заявле...","В компании ""Гамма"" работает мужчина по имени П...",Может ли Петр обратиться в суд по поводу возмо...,"Да, Петр имеет право обратиться в суд с заявле..."
3,Часть 1 / Раздел 1 / Глава 1 / Статья 4.\n##За...,"C1: В компании ""Альфа"" работник Иванов Алексей...","В компании ""Альфа"" работник Иванов Алексей Але...",Может ли Иванов обратиться в суд за нарушение ...,"Да, Иванов может обратиться в суд за нарушение...","В компании ""Бета"" работнику Петрову Ивану Васи...",Может ли Петров отказаться от выполнения работ...,"Да, Петров имеет право отказаться от выполнени...","В компании ""Гамма"" работнику Сидорову Петру Ни...",Может ли Сидоров отказаться от выполнения рабо...,"Да, Сидоров имеет право отказаться от выполнен..."
4,Часть 1 / Раздел 1 / Глава 1 / Статья 5.\n##Тр...,C1: Василий работает в крупной компании и стол...,Василий работает в крупной компании и столкнул...,Какие нормативные правовые акты регулируют тру...,Трудовые отношения и иные непосредственно связ...,Анна работает в маленькой компании и ей отказы...,"Что делать, если работодатель отказывает в опл...",В случае отказа работодателя в оплате сверхуро...,"Иван работает на предприятии, где не соблюдают...",Какие нормативные правовые акты регулируют охр...,Охрана труда регулируется трудовым законодател...
...,...,...,...,...,...,...,...,...,...,...,...
522,Часть 5 / Раздел 13 / Глава 61 / Статья 418.\n...,"C1: В компании ""Альфа"" возник коллективный тру...","В компании ""Альфа"" возник коллективный трудово...",Какие документы должны быть оформлены при расс...,"Действия сторон коллективного трудового спора,...","В компании ""Бета"" произошел коллективный трудо...",Какие органы могут оформлять действия и решени...,"Действия сторон коллективного трудового спора,...","В организации ""Гамма"" произошла забастовка, и ...",Кто может оформлять действия и решения при рас...,"Действия сторон коллективного трудового спора,..."
524,Часть 6 / Раздел 14 / Статья 420.\n##Сроки вве...,"C1: Василий работает в компании, которая была ...","Василий работает в компании, которая была созд...",Какие сроки введения в действие настоящего Код...,"В случае, если компания была создана до 1 февр...","Екатерина работает в новой компании, которая б...",Какие сроки введения в действие настоящего Код...,"В случае, если компания была создана после 1 ф...","Андрей работает в компании, которая была созда...",Какие сроки введени

In [292]:
[[f'{s}{i}'for s in 'CQA'] for i in range(1,4)]

[['C1', 'Q1', 'A1'], ['C2', 'Q2', 'A2'], ['C3', 'Q3', 'A3']]

In [311]:
result_df = result_df[result_df.other_text.isna()]
result_df.drop('other_text', inplace=True, axis=1)
result_df.dropna(inplace=True)

C:\Users\Aydar\AppData\Local\Temp\ipykernel_27028\736619888.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.drop('other_text', inplace=True, axis=1)
C:\Users\Aydar\AppData\Local\Temp\ipykernel_27028\736619888.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.dropna(inplace=True)


In [312]:
ready_to_save_df = melt_modified(result_df, ['document', 'generated_text'], [[f'{s}{i}'for s in 'CQA'] for i in range(1,4)], list('CQA'))

In [313]:
ready_to_save_df

,document,generated_text,index,C,Q,A
0,Часть 1 / Раздел 1 / Глава 1 / Статья 1.\n##Це...,C1: Василий работает в крупной компании и был ...,0,Василий работает в крупной компании и был увол...,Является ли увольнение Василия без объяснения ...,"Нет, увольнение Василия без объяснения причин ..."
1,Часть 1 / Раздел 1 / Глава 1 / Статья 2.\n##Ос...,C1: Василий работает в крупной компании и заме...,1,"Василий работает в крупной компании и заметил,...",Есть ли у Василия право на равную оплату труда...,"Да, у Василия есть право на равную оплату труд..."
2,Часть 1 / Раздел 1 / Глава 1 / Статья 3.\n##За...,"C1: В компании ""Альфа"" работает женщина по име...",2,"В компании ""Альфа"" работает женщина по имени А...",Может ли Анна обратиться в суд по поводу отказ...,"Да, Анна имеет право обратиться в суд с заявле..."
3,Часть 1 / Раздел 1 / Глава 1 / Статья 4.\n##За...,"C1: В компании ""Альфа"" работник Иванов Алексей...",3,"В компании ""Альфа"" работник Иванов Алексей Але...",Может ли Иванов обратиться в суд за нарушение ...,"Да, Иванов может обратиться в суд за нарушение..."
4,Часть 1 / Раздел 1 / Глава 1 / Статья 5.\n##Тр...,C1: Василий работает в крупной компании и стол...,4,Василий работает в крупной компании и столкнул...,Какие нормативные правовые акты регулируют тру...,Трудовые отношения и иные непосредственно связ...
...,...,...,...,...,...,...
1363,Часть 5 / Раздел 13 / Глава 61 / Статья 418.\n...,"C1: В компании ""Альфа"" возник коллективный тру...",522,"В организации ""Гамма"" произошла забастовка, и ...",Кто может оформлять действия и решения при рас...,"Действия сторон коллективного трудового спора,..."
1364,Часть 6 / Раздел 14 / Статья 420.\n##Сроки вве...,"C1: Василий работает в компании, которая была ...",524,"Андрей работает в компании, которая была созда...",Какие сроки введения в действие настоящего Код...,"В случае, если компания была создана до 1 февр..."
1365,Часть 6 / Раздел 14 / Статья 421.\n##Порядок и...,C1: Владимир работает на стройке уже несколько...,525,"Денис работает на предприятии, которое находит...",Может ли работодатель снизить оплату труда раб...,Согласно части первой статьи 133 Трудового Код...
1366,Часть 6 / Раздел 14 / Статья 423.\n##Применени...,C1: Владимир работает в крупной компании и явл...,527,Алексей работает в государственной организации...,Какие законы и нормативные правовые акты должн...,В государственной организации должны применять...


In [316]:
ready_to_save_df.to_json("dataset.json", orient='records')

In [323]:
ready_to_save_df['C'][0]

'Василий работает в крупной компании и был уволен без объяснения причин. Он хочет обратиться к адвокату, чтобы защитить свои трудовые права и выяснить, был ли его увольнение законным.'

In [1]:
import pandas as pd
df = pd.read_json('dataset.json', orient='records')
# df[['C', 'Q', 'A']].to_csv('dataset.csv', index=False)

In [2]:
df

,document,generated_text,index,C,Q,A
0,Часть 1 / Раздел 1 / Глава 1 / Статья 1.\n##Це...,C1: Василий работает в крупной компании и был ...,0,Василий работает в крупной компании и был увол...,Является ли увольнение Василия без объяснения ...,"Нет, увольнение Василия без объяснения причин ..."
1,Часть 1 / Раздел 1 / Глава 1 / Статья 2.\n##Ос...,C1: Василий работает в крупной компании и заме...,1,"Василий работает в крупной компании и заметил,...",Есть ли у Василия право на равную оплату труда...,"Да, у Василия есть право на равную оплату труд..."
2,Часть 1 / Раздел 1 / Глава 1 / Статья 3.\n##За...,"C1: В компании ""Альфа"" работает женщина по име...",2,"В компании ""Альфа"" работает женщина по имени А...",Может ли Анна обратиться в суд по поводу отказ...,"Да, Анна имеет право обратиться в суд с заявле..."
3,Часть 1 / Раздел 1 / Глава 1 / Статья 4.\n##За...,"C1: В компании ""Альфа"" работник Иванов Алексей...",3,"В компании ""Альфа"" работник Иванов Алексей Але...",Может ли Иванов обратиться в суд за нарушение ...,"Да, Иванов может обратиться в суд за нарушение..."
4,Часть 1 / Раздел 1 / Глава 1 / Статья 5.\n##Тр...,C1: Василий работает в крупной компании и стол...,4,Василий работает в крупной компании и столкнул...,Какие нормативные правовые акты регулируют тру...,Трудовые отношения и иные непосредственно связ...
...,...,...,...,...,...,...
1363,Часть 5 / Раздел 13 / Глава 61 / Статья 418.\n...,"C1: В компании ""Альфа"" возник коллективный тру...",522,"В организации ""Гамма"" произошла забастовка, и ...",Кто может оформлять действия и решения при рас...,"Действия сторон коллективного трудового спора,..."
1364,Часть 6 / Раздел 14 / Статья 420.\n##Сроки вве...,"C1: Василий работает в компании, которая была ...",524,"Андрей работает в компании, которая была созда...",Какие сроки введения в действие настоящего Код...,"В случае, если компания была создана до 1 февр..."
1365,Часть 6 / Раздел 14 / Статья 421.\n##Порядок и...,C1: Владимир работает на стройке уже несколько...,525,"Денис работает на предприятии, которое находит...",Может ли работодатель снизить оплату труда раб...,Согласно части первой статьи 133 Трудового Код...
1366,Часть 6 / Раздел 14 / Статья 423.\n##Применени...,C1: Владимир работает в крупной компании и явл...,527,Алексей работает в государственной организации...,Какие законы и нормативные правовые акты должн...,В государственной организации должны применять...


In [4]:
def format_instruct_dataset_record(record: pd.Series):
    return f"[INST]{record['C']} {record['Q']}[/INST]{record['A']}"

In [331]:
all_records_joined = "<s>".join(ready_to_save_df.apply(format_instruct_dataset_record, axis=1))
with open('case_question_answer_dataset.txt', 'w', encoding='utf8') as f:
    f.write(all_records_joined)

In [2]:
import pandas as pd
df = pd.read_json('dataset.json', orient='records')
df

,document,generated_text,index,C,Q,A
0,Часть 1 / Раздел 1 / Глава 1 / Статья 1.\n##Це...,C1: Василий работает в крупной компании и был ...,0,Василий работает в крупной компании и был увол...,Является ли увольнение Василия без объяснения ...,"Нет, увольнение Василия без объяснения причин ..."
1,Часть 1 / Раздел 1 / Глава 1 / Статья 2.\n##Ос...,C1: Василий работает в крупной компании и заме...,1,"Василий работает в крупной компании и заметил,...",Есть ли у Василия право на равную оплату труда...,"Да, у Василия есть право на равную оплату труд..."
2,Часть 1 / Раздел 1 / Глава 1 / Статья 3.\n##За...,"C1: В компании ""Альфа"" работает женщина по име...",2,"В компании ""Альфа"" работает женщина по имени А...",Может ли Анна обратиться в суд по поводу отказ...,"Да, Анна имеет право обратиться в суд с заявле..."
3,Часть 1 / Раздел 1 / Глава 1 / Статья 4.\n##За...,"C1: В компании ""Альфа"" работник Иванов Алексей...",3,"В компании ""Альфа"" работник Иванов Алексей Але...",Может ли Иванов обратиться в суд за нарушение ...,"Да, Иванов может обратиться в суд за нарушение..."
4,Часть 1 / Раздел 1 / Глава 1 / Статья 5.\n##Тр...,C1: Василий работает в крупной компании и стол...,4,Василий работает в крупной компании и столкнул...,Какие нормативные правовые акты регулируют тру...,Трудовые отношения и иные непосредственно связ...
...,...,...,...,...,...,...
1363,Часть 5 / Раздел 13 / Глава 61 / Статья 418.\n...,"C1: В компании ""Альфа"" возник коллективный тру...",522,"В организации ""Гамма"" произошла забастовка, и ...",Кто может оформлять действия и решения при рас...,"Действия сторон коллективного трудового спора,..."
1364,Часть 6 / Раздел 14 / Статья 420.\n##Сроки вве...,"C1: Василий работает в компании, которая была ...",524,"Андрей работает в компании, которая была созда...",Какие сроки введения в действие настоящего Код...,"В случае, если компания была создана до 1 февр..."
1365,Часть 6 / Раздел 14 / Статья 421.\n##Порядок и...,C1: Владимир работает на стройке уже несколько...,525,"Денис работает на предприятии, которое находит...",Может ли работодатель снизить оплату труда раб...,Согласно части первой статьи 133 Трудового Код...
1366,Часть 6 / Раздел 14 / Статья 423.\n##Применени...,C1: Владимир работает в крупной компании и явл...,527,Алексей работает в государственной организации...,Какие законы и нормативные правовые акты должн...,В государственной организации должны применять...


In [18]:
df.A[df.A.str.contains('"')].iloc[25]

'Работодатель нарушает принципы социального партнерства, в частности, не соблюдает обязательность выполнения коллективного договора и соглашений. "Ответственность сторон, их представителей за невыполнение по их вине коллективных договоров, соглашений" (Статья 24 Трудового Кодекса РФ).'

In [46]:
lcs('asdfghj', 'sdfg')

'sdfg'

In [3]:
import re
# from difflib import SequenceMatcher
# def longest_common_substring(string1, string2):
#     matcher = SequenceMatcher(None, string1, string2)
#     match = matcher.find_longest_match(0, len(string1), 0, len(string2))
#     longest_substring = string1[match.a: match.a + match.size]
#     return longest_substring


def remove_ellipses_and_square_brackets(text):
    # Remove ellipses
    text = re.sub(r'\.{3}', '', text)
    # Remove text within square brackets
    text = re.sub(r'\[\.?\]', '', text)
    return text

# # Example usage:
# input_text = "This is some text [...] with ellipses... [inside brackets]."
# output_text = remove_ellipses_and_square_brackets(input_text)
# print(output_text)

def replace_spaces(input_string):
    # Replace any sequence of space symbols with a single space
    result = re.sub(r'\s+', ' ', input_string)
    return result


def extract_quoted_text(input_string):
    pattern = r'"([^"]*)"'
    matches = re.findall(pattern, input_string)
    for i in range(len(matches)):
        matches[i] = remove_ellipses_and_square_brackets(matches[i]).strip()
        matches[i] = replace_spaces(matches[i])
    return matches

def has_citation(record):
    answer = record.A
    
    document = replace_spaces(record.document)
    quoted_texts = extract_quoted_text(answer)
    for quote in quoted_texts:
#         if len(longest_common_substring(document.lower(), quote.lower())) / len(quote.lower()) > 0.8:
        if quote.lower() in document.lower():
            return True
    return False
    

In [4]:
with_cite = df.apply(has_citation, axis=1)
with_cite.sum()

894

In [128]:
df[~with_cite].A.map(extract_quoted_text)

0       [увольнение работника по инициативе работодате...
1       [право работников на объединение для защиты св...
2                                    [[place quote here]]
4                                    [[place quote here]]
6       [Работник вправе получать заработную плату не ...
                              ...                        
1356                                                   []
1358                                 [[place quote here]]
1361    [Представители работодателя (представители раб...
1362                                                   []
1365    [Работник имеет право на оплату труда не ниже ...
Name: A, Length: 474, dtype: object

In [23]:
df_with_cite = df[with_cite]
df_with_cite = df_with_cite.rename(columns={'index': "article_number_trudkod"})
df_with_cite.index.name = 'index'
test_df = df_with_cite.sample(frac=0.2, random_state=123)
train_df = df_with_cite.drop(test_df.index)

In [25]:
test_df.to_csv('qa_test.csv')
train_df.to_csv('qa_train.csv')

In [24]:
df_with_cite

,document,generated_text,article_number_trudkod,C,Q,A
index,,,,,,
3,Часть 1 / Раздел 1 / Глава 1 / Статья 4.\n##За...,"C1: В компании ""Альфа"" работник Иванов Алексей...",3,"В компании ""Альфа"" работник Иванов Алексей Але...",Может ли Иванов обратиться в суд за нарушение ...,"Да, Иванов может обратиться в суд за нарушение..."
5,Часть 1 / Раздел 1 / Глава 1 / Статья 6.\n##Ра...,"C1: В компании ""Альфа"" работник по имени Иван ...",5,"В компании ""Альфа"" работник по имени Иван обра...",Какие меры защиты предусмотрены законом для ра...,"Работникам, которые сталкиваются с нарушением ..."
8,Часть 1 / Раздел 1 / Глава 1 / Статья 9.\n##Ре...,"C1: В компании ""Альфа"" работник Иванов обратил...",8,"В компании ""Альфа"" работник Иванов обратился к...","Могут ли условия, ограничивающие права работни...","Нет, условия, ограничивающие права работника, ..."
9,Часть 1 / Раздел 1 / Глава 1 / Статья 10.\n##Т...,C1: Владимир работает в крупной международной ...,9,Владимир работает в крупной международной комп...,Может ли Владимир ссылаться на международные д...,"Да, Владимир может ссылаться на международные ..."
11,Часть 1 / Раздел 1 / Глава 1 / Статья 12.\n##Д...,"C1: Василий работает в компании ""Альфа"" с 2010...",11,"Василий работает в компании ""Альфа"" с 2010 год...",Какие права и обязанности применяются к Васили...,Права и обязанности Василия в отношении локаль...
...,...,...,...,...,...,...
1360,Часть 5 / Раздел 13 / Глава 61 / Статья 415.\n...,"C1: В компании ""Альфа"" произошел коллективный ...",519,"В компании ""Гамма"" произошел коллективный труд...","Может ли работодатель уволить работников, кото...",Работодатель не имеет права увольнять работник...
1363,Часть 5 / Раздел 13 / Глава 61 / Статья 418.\n...,"C1: В компании ""Альфа"" возник коллективный тру...",522,"В организации ""Гамма"" произошла забастовка, и ...",Кто может оформлять действия и решения при рас...,"Действия сторон коллективного трудового спора,..."
1364,Часть 6 / Раздел 14 / Статья 420.\n##Сроки вве...,"C1: Василий работает в компании, которая была ...",524,"Андрей работает в компании, которая была созда...",Какие сроки введения в действие настоящего Код...,"В случае, если компания была создана до 1 февр..."
